In [43]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import (StandardScaler, MinMaxScaler, RobustScaler,
                                   PowerTransformer, QuantileTransformer, LabelEncoder)
from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression
from sklearn.ensemble import IsolationForest
from sklearn.metrics import accuracy_score, r2_score
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

from lightgbm import LGBMClassifier
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score
from scipy import stats
from scipy.spatial.distance import pdist, squareform
from persiantools.jdatetime import JalaliDate
from datetime import datetime
import lightgbm as lgb
import re
import warnings
warnings.filterwarnings('ignore')

In [44]:
divar_df = pd.read_parquet('Divar.parquet')

In [45]:
df = divar_df.copy()
original_shape = df.shape
print(f"Original dataset: {original_shape[0]:,} rows × {original_shape[1]} columns")

Original dataset: 1,000,000 rows × 61 columns


In [46]:
# Column definitions for real estate domain
target_columns = ['price_value', 'rent_value',
                  'credit_value', 'transformable_price']

boolean_columns = ['rent_to_single', 'has_business_deed', 'has_balcony',
                   'has_elevator', 'has_electricity', 'has_gas', 'has_security_guard',
                   'has_warehouse', 'has_parking', 'is_rebuilt', 'has_water',
                   'has_barbecue', 'has_pool', 'has_jacuzzi', 'has_sauna',
                   'rent_credit_transform', 'transformable_price']
numerical_columns = [
    'rent_value', 'price_value', 'credit_value',
    'transformable_credit', 'transformed_credit', 'transformable_rent', 'transformed_rent',
    'land_size', 'building_size', 'floor', 'rooms_count', 'total_floors_count',
    'unit_per_floor', 'construction_year', 'regular_person_capacity',
    'extra_person_capacity', 'cost_per_extra_person',
    'rent_price_on_regular_days', 'rent_price_on_special_days',
    'rent_price_at_weekends', 'month_created', 'seaseon_created', 'year_created'
]

price_columns = ['price_value', 'rent_value', 'credit_value', 'transformable_price',
                 'transformable_credit', 'transformed_credit', 'transformable_rent',
                 'transformed_rent', 'cost_per_extra_person','rent_price_on_regular_days', 
                 'rent_price_on_special_days', 'rent_price_at_weekends',]

size_columns = ['land_size', 'building_size']

location_columns = ['cat2_slug', 'cat3_slug',
                    'city_slug', 'neighborhood_slug']

one_hot_colums = ['has_cooling_system', 'has_restroom',
                  'has_heating_system', 'has_warm_water_provider']

categorical_columns = ['user_type', 'rent_mode', 'credit_mode'
                       'cat2_slug', 'cat3_slug', 'city_slug', 
                       'deed_type', 'building_direction','floor_material', 
                       'property_type', 'neighborhood_slug'
                       ]

amenity_columns = [
    'has_balcony', 'has_elevator', 'has_parking', 'has_warehouse', 'price_mode',
    'has_water', 'has_gas', 'has_electricity', 'has_pool', 'has_jacuzzi',
    'has_sauna', 'has_barbecue', 'has_security_guard'
]

property_columns = ['property_type','building_direction',
                    'floor_material', 'deed_type']

# =============================================================================
# STEP 2:  DATA CLEANING AND TRANSFORMATION
# =============================================================================

In [47]:
# Drop obviously irrelevant columns
drop_cols = ['Unnamed: 0', 'location_radius', 'location_latitude', 'location_longitude', 'title', 'description' ]
deleted_colums = df[drop_cols]
df.drop(columns=[col for col in drop_cols if col in df.columns], inplace=True)


In [48]:
# Persian number conversion function
def convert_persian_numbers(text):
    if pd.isna(text):
        return text
    persian_digits = '۰۱۲۳۴۵۶۷۸۹'
    english_digits = '0123456789'
    for p, e in zip(persian_digits, english_digits):
        text = str(text).replace(p, e)
    return text

In [49]:
# Apply Persian conversion to relevant columns
persian_cols = ['construction_year', 'total_floors_count', 'floor', 'rooms_count']
for col in persian_cols:
    if col in df.columns:
        df[col] = df[col].apply(convert_persian_numbers)


In [50]:
#  rooms count mapping
rooms_mapping = {
    'بدون اتاق': 0, 'یک': 1, 'دو': 2, 'سه': 3, 'چهار': 4,
    'پنج یا بیشتر': 5, '5+': 5, 'استودیو': 0
}
if 'rooms_count' in df.columns:
    df['rooms_count'] = df['rooms_count'].map(rooms_mapping).astype('Int8')
 

In [51]:
# Construction year cleaning with validation
if 'construction_year' in df.columns:
    df['construction_year'] = pd.to_numeric(df['construction_year'].str.extract(r'(\d+)')[0], errors='coerce')



In [52]:
# Floor and building floors validation
for col in ['floor', 'total_floors_count', 'extra_person_capacity']:
    if col in df.columns:
        df[col] = df[col].replace({'30+': 31, 'more_than_30': 31, 'unselect': 1})
        df[col] = pd.to_numeric(df[col], errors='coerce')


In [53]:
# Unit per floor cleaning
if 'unit_per_floor' in df.columns:
    df['unit_per_floor'] = df['unit_per_floor'].replace({
        'more_than_8': 9, 'unselect': 1
    })
    df['unit_per_floor'] = pd.to_numeric(df['unit_per_floor'], errors='coerce')

In [54]:
# Persian text mapping for categorical columns
persian_mappings = {
    'user_type': {'مشاور املاک': 'agent', 'شخصی': 'personal'},
    'rent_mode': {'مقطوع': 'fixed', 'مجانی': 'free', 'توافقی': 'negotiable'},
    'price_mode': {'مقطوع': 'fixed', 'مجانی': 'free', 'توافقی': 'negotiable'},
    'credit_mode': {'مقطوع': 'fixed', 'مجانی': 'free', 'توافقی': 'negotiable'}
}

for col, mapping in persian_mappings.items():
    if col in df.columns:
        df[col] = df[col].map(mapping)

In [55]:
# Boolean column standardization
bool_map = {True: 1, False: 0, 'true': 1, 'false': 0, 'yes': 1, 'no': 0, 'unselect': 0}
for col in boolean_columns:
    if col in df.columns:
        df[col] = df[col].map(bool_map).astype('Int8')

# =============================================================================
# STEP 3:  OUTLIER DETECTION AND HANDLING
# =============================================================================

In [56]:
def detect_outliers_multiple_methods(series, methods=['iqr', 'zscore', 'isolation']):
    """Detect outliers using multiple methods for robust identification"""
    outliers = pd.Series(False, index=series.index)

    if 'iqr' in methods:
        Q1 = series.quantile(0.25)
        Q3 = series.quantile(0.75)
        IQR = Q3 - Q1
        iqr_outliers = (series < Q1 - 2.5 * IQR) | (series > Q3 + 2.5 * IQR)
        outliers |= iqr_outliers

    if 'zscore' in methods and len(series) > 3:
        z_scores = np.abs(stats.zscore(series, nan_policy='omit'))
        zscore_outliers = z_scores > 3.5
        outliers |= zscore_outliers

    if 'isolation' in methods and len(series) > 50:
        try:
            iso_forest = IsolationForest(contamination=0.05, random_state=42)
            iso_outliers = iso_forest.fit_predict(series.values.reshape(-1, 1)) == -1
            outliers |= iso_outliers
        except:
            pass

    return outliers

# =============================================================================
# STEP 4: MISSING VALUE IMPUTATION
# =============================================================================

In [58]:

# Features for training (exclude boolean targets except first)
features = df.drop(columns=boolean_columns).copy()

# Encode categorical columns
for col in features.select_dtypes('object').columns:
    features[col] = features[col].astype('category').cat.codes

# Fill numeric NaNs
for col in features.select_dtypes('number').columns:
    features[col] = features[col].fillna(features[col].median())

for target in boolean_columns:
    print(f"Filling: {target}")
    mask_train = df[target].notna()
    if mask_train.sum() == 0:
        continue

    X_train, y_train = features[mask_train], df.loc[mask_train, target].astype(int)
    X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

    train_data = lgb.Dataset(X_tr, label=y_tr)
    val_data = lgb.Dataset(X_val, label=y_val, reference=train_data)

    model = lgb.train(
        {'objective':'binary','metric':'binary_error','boosting_type':'gbdt',
         'learning_rate':0.05,'num_leaves':31,'verbose':-1,'seed':42},
        train_data, valid_sets=[val_data]
    )

    mask_missing = df[target].isna()
    if mask_missing.any():
        df.loc[mask_missing, target] = (model.predict(features[mask_missing]) > 0.5).astype(int)

    acc = accuracy_score(y_val, (model.predict(X_val) > 0.5).astype(int))
    print(f"Validation Accuracy: {acc * 100:.1f}")


Filling: rent_to_single
Validation Accuracy: 100.0
Filling: has_business_deed
Validation Accuracy: 94.4
Filling: has_balcony
Validation Accuracy: 91.8
Filling: has_elevator
Validation Accuracy: 91.5
Filling: has_electricity
Validation Accuracy: 87.8
Filling: has_gas
Validation Accuracy: 87.7
Filling: has_security_guard
Validation Accuracy: 93.2
Filling: has_warehouse
Validation Accuracy: 81.7
Filling: has_parking
Validation Accuracy: 83.3
Filling: is_rebuilt
Validation Accuracy: 83.4
Filling: has_water
Validation Accuracy: 87.5
Filling: has_barbecue
Validation Accuracy: 90.1
Filling: has_pool
Validation Accuracy: 93.9
Filling: has_jacuzzi
Validation Accuracy: 95.9
Filling: has_sauna
Validation Accuracy: 97.8
Filling: rent_credit_transform
Validation Accuracy: 99.8
Filling: transformable_price
Validation Accuracy: 99.7


In [59]:
# Apply outlier detection to price and size columns
outlier_summary = {}

for col in price_columns + size_columns:
    if col in df.columns and df[col].notna().sum() > 100:
        # Ensure column is numeric
        df[col] = pd.to_numeric(df[col], errors='coerce')

        # Work only with valid numeric data
        valid_data = df[col].dropna()
        if valid_data.empty:
            continue  # skip if nothing usable

        # Detect outliers
        outliers = detect_outliers_multiple_methods(valid_data)
        outlier_count = outliers.sum()
        outlier_percentage = (outlier_count / len(valid_data)) * 100

        # Save summary
        outlier_summary[col] = {
            'count': outlier_count,
            'percentage': outlier_percentage
        }

        # Cap extreme outliers instead of removing them
        if outlier_percentage > 1:  # only if significant
            lower_cap = df[col].quantile(0.01)
            upper_cap = df[col].quantile(0.99)
            df[col] = df[col].clip(lower=lower_cap, upper=upper_cap)

        print(f"  {col}: {outlier_count} outliers ({outlier_percentage:.1f}%) - capped")


  price_value: 36643 outliers (6.4%) - capped
  rent_value: 24184 outliers (6.9%) - capped
  credit_value: 18802 outliers (5.3%) - capped
  transformable_price: 243276 outliers (24.3%) - capped
  transformable_credit: 18802 outliers (5.3%) - capped
  transformed_credit: 4375 outliers (6.0%) - capped
  transformable_rent: 24183 outliers (6.9%) - capped
  transformed_rent: 4038 outliers (5.6%) - capped
  cost_per_extra_person: 512 outliers (5.0%) - capped
  rent_price_on_regular_days: 802 outliers (4.4%) - capped
  rent_price_on_special_days: 524 outliers (5.0%) - capped
  rent_price_at_weekends: 611 outliers (4.5%) - capped
  land_size: 13287 outliers (7.1%) - capped
  building_size: 89075 outliers (9.1%) - capped


In [60]:
# Separate columns by missing value percentage for different strategies
missing_analysis = df.isnull().sum() / len(df) * 100
low_missing = missing_analysis[missing_analysis <= 5].index.tolist()
medium_missing = missing_analysis[(missing_analysis > 5) & (missing_analysis <= 30)].index.tolist()
high_missing = missing_analysis[missing_analysis > 30].index.tolist()

print(f"Low missing (<5%): {len(low_missing)} columns")
print(f"Medium missing (5-30%): {len(medium_missing)} columns")
print(f"High missing (>30%): {len(high_missing)} columns")


Low missing (<5%): 22 columns
Medium missing (5-30%): 2 columns
High missing (>30%): 31 columns


In [61]:
# Check missing values for all columns
missing_values = df.isnull().sum()
missing_percentage = (df.isnull().sum() / len(df)) * 100

# Create missing values summary
missing_summary = pd.DataFrame({
    'missing_count': missing_values,
    'missing_percentage': missing_percentage.round(2)
})

# Show columns with missing values
columns_with_missing = missing_summary[missing_summary['missing_count'] > 0]
print("Columns with missing values:")
print(columns_with_missing.sort_values('missing_percentage', ascending=False))

# Show columns without missing values
columns_without_missing = missing_summary[missing_summary['missing_count'] == 0]
print(f"\nColumns without missing values: {len(columns_without_missing)}")

# Total summary
print(f"\nTotal columns: {len(df.columns)}")
print(f"Columns with missing values: {len(columns_with_missing)}")
print(f"Columns without missing values: {len(columns_without_missing)}")
print(f"Overall missing percentage: {df.isnull().sum().sum() / (len(df) * len(df.columns)) * 100:.2f}%")

Columns with missing values:
                            missing_count  missing_percentage
cost_per_extra_person              989759               98.98
rent_price_on_special_days         989537               98.95
rent_price_at_weekends             986449               98.64
rent_price_on_regular_days         981932               98.19
extra_person_capacity              975991               97.60
property_type                      972943               97.29
regular_person_capacity            970130               97.01
transformed_credit                 927591               92.76
transformed_rent                   927591               92.76
rent_type                          896039               89.60
land_size                          813604               81.36
deed_type                          746542               74.65
user_type                          711118               71.11
unit_per_floor                     697717               69.77
total_floors_count                 695648

In [62]:
# Strategy 1: Simple imputation for low missing
# Impute low missing numerical columns
continus_numeric = price_columns + size_columns

all_numric  = df.select_dtypes(include=['number']).columns.tolist()
for col in low_missing:
    if col in all_numric:
        if col in continus_numeric:  # Continuous price-like columns
            # Use median for price columns (robust to outliers)
            imputer = SimpleImputer(strategy='median')
            df[col] = imputer.fit_transform(df[[col]]).ravel()
            print(f"Imputed {col} with median: {imputer.statistics_[0]:.2f}")
        else:
            # Use mean for other numerical columns
            imputer = SimpleImputer(strategy='mean')
            df[col] = imputer.fit_transform(df[[col]]).ravel()
            print(f"Imputed {col} with mean: {imputer.statistics_[0]:.2f}")
    
    elif col in categorical_columns:
        # Use mode for categorical columns
        imputer = SimpleImputer(strategy='most_frequent')
        df[col] = imputer.fit_transform(df[[col]]).ravel()
        print(f"Imputed {col} with mode: {imputer.statistics_[0]}")

Imputed cat3_slug with mode: apartment-sell
Imputed city_slug with mode: tehran
Imputed rent_to_single with mean: 1.00
Imputed rent_credit_transform with mean: 0.15
Imputed transformable_price with median: 0.00
Imputed building_size with median: 103.00
Imputed has_business_deed with mean: 0.48
Imputed has_balcony with mean: 0.83
Imputed has_elevator with mean: 0.60
Imputed has_warehouse with mean: 0.78
Imputed has_parking with mean: 0.75
Imputed is_rebuilt with mean: 0.40
Imputed has_water with mean: 0.36
Imputed has_electricity with mean: 0.35
Imputed has_gas with mean: 0.34
Imputed has_security_guard with mean: 0.29
Imputed has_barbecue with mean: 0.29
Imputed has_pool with mean: 0.29
Imputed has_jacuzzi with mean: 0.29
Imputed has_sauna with mean: 0.29


In [ ]:
# # Strategy 2: KNN imputation for medium missing numerical columns
# medium_numerical = [col for col in medium_missing if col in df.select_dtypes(include=[np.number]).columns]
# if medium_numerical:
#     print(f"Applying KNN imputation to {len(medium_numerical)} numerical columns...")
#     knn_imputer = KNNImputer(n_neighbors=5, weights='distance')
#     df[medium_numerical] = knn_imputer.fit_transform(df[medium_numerical])

# =============================================================================
# STEP :  CATEGORICAL ENCODING
# =============================================================================

In [64]:

# Identify categorical columns
categorical_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()

# Target encoding for high-cardinality location columns (if target available)
high_card_cols = ['city_slug', 'neighborhood_slug']
target_col = 'transformable_price'  # Primary target
if target_col in df.columns:
    for col in high_card_cols:
        if col in df.columns and col in categorical_cols and col not in one_hot_colums:
            # Calculate mean target value for each category
            target_mean = df.groupby(col)[target_col].mean()
            global_mean = df[target_col].mean()

            # Add smoothing to prevent overfitting
            category_counts = df[col].value_counts()
            smoothing = 100  # Smoothing parameter
            smoothed_means = (target_mean * category_counts + global_mean * smoothing) / (category_counts + smoothing)

            df[f'{col}_target_encoded'] = df[col].map(smoothed_means).fillna(global_mean).astype('float32')
            categorical_cols.remove(col) if col in categorical_cols else None
            
            
# Label encoding for remaining categorical columns
label_encoders = {}
for col in categorical_cols:
    if col in df.columns:
        le = LabelEncoder()
        # Handle missing values
        df[col] = df[col].astype(str).fillna('Unknown')
        df[col] = le.fit_transform(df[col])
        label_encoders[col] = le

        # Convert to appropriate integer type
        if len(le.classes_) < 128:
            df[col] = df[col].astype('Int8')
        elif len(le.classes_) < 32768:
            df[col] = df[col].astype('Int16')

print(f"Encoded {len(categorical_cols)} categorical columns")

Encoded 16 categorical columns


In [68]:
from sklearn.preprocessing import OneHotEncoder

# One-hot encode specified columns using sklearn
one_hot_columns = ['has_cooling_system', 'has_restroom', 'has_heating_system', 'has_warm_water_provider']

for col in one_hot_columns:
    if col in df.columns:
        # Convert to string and handle NaN
        col_data = df[col].astype(str)
        
        # Initialize and fit one-hot encoder
        ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
        encoded_data = ohe.fit_transform(col_data.values.reshape(-1, 1))
        
        # Create column names
        feature_names = ohe.get_feature_names_out([col])
        
        # Create DataFrame with encoded columns
        encoded_df = pd.DataFrame(encoded_data, columns=feature_names, index=df.index)
        
        # Drop original column and concatenate encoded columns
        df.drop(col, axis=1, inplace=True)
        df = pd.concat([df, encoded_df], axis=1)

In [78]:
# Check for remaining object columns
object_cols = df.select_dtypes(include=['object']).columns.tolist()

if object_cols:
    print("Object columns remaining:", object_cols)
    print("Number of object columns:", len(object_cols))
else:
    print("No object columns left - all data is numeric")

No object columns left - all data is numeric


In [76]:
# Label encode high cardinality columns but keep missing values as NaN
high_card_cols = ['city_slug', 'neighborhood_slug']

for col in high_card_cols:
    # Create temporary copy without missing values for fitting
    non_null_mask = df[col].notna()
    non_null_values = df.loc[non_null_mask, col]
    
    # Fit LabelEncoder on non-null values only
    le = LabelEncoder()
    le.fit(non_null_values.astype(str))
    
    # Transform only non-null values, keep missing as NaN
    df.loc[non_null_mask, col] = le.transform(non_null_values.astype(str))
    
    # Convert to appropriate integer type
    unique_count = len(le.classes_)
    if unique_count < 128:
        df[col] = df[col].astype('Int8')
    elif unique_count < 32768:
        df[col] = df[col].astype('Int16')
    else:
        df[col] = df[col].astype('Int32')

In [77]:
df['neighborhood_slug'].unique()

<IntegerArray>
[ 669,  363, 1121,  274,  283,  675, <NA>,  221,  319,  369,
 ...
   12, 1029,  225, 1080,  148,  890,  870, 1024,  473,  469]
Length: 1189, dtype: Int16

In [ ]:
# Strategy 4: Handle high missing columns


for col in high_missing:
    print(f"\n⚠️  Processing high missing column: {col}")
    
    missing_mask = df[col].isnull()
    complete_data = df[~missing_mask].copy()
    
    if len(complete_data) < 50:  # Very few samples available
        print(f"Very few complete samples ({len(complete_data)}) for {col}, using simple imputation")
        
        if col in categorical_columns or col in boolean_columns:
            mode_val = complete_data[col].mode()[0] if len(complete_data) > 0 else df[col].mode()[0]
            df.loc[missing_mask, col] = mode_val
            print(f"Used mode: {mode_val}")
        else:
            median_val = complete_data[col].median() if len(complete_data) > 0 else df[col].median()
            df.loc[missing_mask, col] = median_val
            print(f"Used median: {median_val:.2f}")
        continue
    
    if col in categorical_columns or col in boolean_columns:
        print(f"Using RandomForestClassifier for high missing: {col}")
        
        features = [c for c in df.columns if c != col and df[c].isnull().mean() < 0.2]
        X = complete_data[features]
        y = complete_data[col]
        
        if len(X) < 100:
            # Use cross-validation for small datasets
            
            model = RandomForestClassifier(n_estimators=50, random_state=42)
            scores = cross_val_score(model, X, y, cv=3, scoring='accuracy')
            avg_acc = scores.mean()
            print(f"Cross-val accuracy for {col}: {avg_acc:.4f}")
            
            if avg_acc >= 0.7:  # Lower threshold for high missing
                model.fit(X, y)
                missing_data = df[missing_mask][features].copy()
                predictions = model.predict(missing_data)
                df.loc[missing_mask, col] = predictions
                print(f"✅ Imputed with accuracy: {avg_acc:.4f}")
            else:
                mode_val = complete_data[col].mode()[0]
                df.loc[missing_mask, col] = mode_val
                print(f"Used mode fallback: {mode_val}")
        else:
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
            model = RandomForestClassifier(n_estimators=50, random_state=42)
            model.fit(X_train, y_train)
            
            y_pred = model.predict(X_test)
            acc = accuracy_score(y_test, y_pred)
            print(f"Accuracy for {col}: {acc:.4f}")
            
            if acc >= 0.7:
                missing_data = df[missing_mask][features].copy()
                predictions = model.predict(missing_data)
                df.loc[missing_mask, col] = predictions
                print(f"✅ Imputed with accuracy: {acc:.4f}")
            else:
                mode_val = complete_data[col].mode()[0]
                df.loc[missing_mask, col] = mode_val
                print(f"Used mode fallback: {mode_val}")
    
    elif col in e:  # Continuous price columns
        print(f"Using RandomForestRegressor for high missing price: {col}")
        
        features = [c for c in df.columns if c != col and df[c].isnull().mean() < 0.2]
        X = complete_data[features]
        y = complete_data[col]
        
        if len(X) < 100:
            from sklearn.model_selection import cross_val_score
            model = RandomForestRegressor(n_estimators=50, random_state=42)
            scores = cross_val_score(model, X, y, cv=3, scoring='r2')
            avg_r2 = scores.mean()
            print(f"Cross-val R2 for {col}: {avg_r2:.4f}")
            
            if avg_r2 >= 0.6:
                model.fit(X, y)
                missing_data = df[missing_mask][features].copy()
                predictions = model.predict(missing_data)
                df.loc[missing_mask, col] = predictions
                print(f"✅ Imputed with R2: {avg_r2:.4f}")
            else:
                median_val = complete_data[col].median()
                df.loc[missing_mask, col] = median_val
                print(f"Used median fallback: {median_val:.2f}")
        else:
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
            model = RandomForestRegressor(n_estimators=50, random_state=42)
            model.fit(X_train, y_train)
            
            y_pred = model.predict(X_test)
            r2 = r2_score(y_test, y_pred)
            print(f"R2 for {col}: {r2:.4f}")
            
            if r2 >= 0.6:
                missing_data = df[missing_mask][features].copy()
                predictions = model.predict(missing_data)
                df.loc[missing_mask, col] = predictions
                print(f"✅ Imputed with R2: {r2:.4f}")
            else:
                median_val = complete_data[col].median()
                df.loc[missing_mask, col] = median_val
                print(f"Used median fallback: {median_val:.2f}")
    
    else:  # Other numerical columns
        print(f"Using RandomForestRegressor for high missing numerical: {col}")
        
        features = [c for c in df.columns if c != col and df[c].isnull().mean() < 0.2]
        X = complete_data[features]
        y = complete_data[col]
        
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        model = RandomForestRegressor(n_estimators=50, random_state=42)
        model.fit(X_train, y_train)
        
        y_pred = model.predict(X_test)
        r2 = r2_score(y_test, y_pred)
        print(f"R2 for {col}: {r2:.4f}")
        
        if r2 >= 0.6:
            missing_data = df[missing_mask][features].copy()
            predictions = model.predict(missing_data)
            df.loc[missing_mask, col] = predictions
            print(f"✅ Imputed with R2: {r2:.4f}")
        else:
            median_val = complete_data[col].median()
            df.loc[missing_mask, col] = median_val
            print(f"Used median fallback: {median_val:.2f}")



⚠️  Processing high missing column: neighborhood_slug
Using RandomForestClassifier for high missing: neighborhood_slug


# =============================================================================
# STEP 5:  FEATURE ENGINEERING
# =============================================================================

In [ ]:
# Time-based features from Persian date
if 'created_at_month' in df.columns:
    df['created_at_month'] = pd.to_datetime(
        df['created_at_month'], errors='coerce')
    
    # Convert to JalaliDate and extract month directly
    df['shamsi_date'] = df['created_at_month'].apply(
        lambda x: JalaliDate(x) if pd.notna(x) else None)
    
    
    df['month_created'] = df['shamsi_date'].apply(
        lambda x: x.month if x is not None else None
        
    ).astype('Int8')
    
    df['year_created'] = df['shamsi_date'].apply(
        lambda x: x.year if x is not None else None
    ).astype('Int16')
    
    # Create season from month
    df['season_created'] = ((df['month_created'] - 1) // 3 + 1).astype('Int8')    
    
   
    df.drop('created_at_month', axis=1, inplace=True)

In [ ]:
# Building age and condition features
if 'construction_year' in df.columns:
    current_year = 1403  # Current Shamsi year
    df['building_age'] = (current_year - df['construction_year']).clip(0, 100).astype('Int8')
   
    # Age categories for non-linear relationships
    df['age_category'] = pd.cut(df['building_age'],
                               bins=[0, 2, 5, 9, 15, 100],
                               labels=['New', 'Modern', 'Mature', 'Old', 'Very_Old']).astype(str)

    # Building condition score based on age
    df['condition_score'] = np.where(df['building_age'] <= 2, 5,
                           np.where(df['building_age'] <= 5, 4,
                           np.where(df['building_age'] <= 9, 3,
                           np.where(df['building_age'] <= 50, 2, 1)))).astype('Int8')
    
    df.drop('construction_year', axis=1, inplace=True, errors='ignore')


    # Encode age_category with ordinal mapping
    if 'age_category' in df.columns:
        age_category_mapping = {'New': 5, 'Modern': 4, 'Mature': 3, 'Old': 2, 'Very_Old': 1}
        df['age_category_encoded'] = df['age_category'].map(age_category_mapping)
        df['age_category_encoded'].fillna(df['age_category_encoded'].median(), inplace=True)
        df['age_category_encoded'] = df['age_category_encoded'].astype('int8')
        df.drop('age_category', axis=1, inplace=True, errors='ignore')    


In [ ]:
# Size and space utilization features
if all(col in df.columns for col in ['building_size', 'land_size']):
    df['building_to_land_ratio'] = (df['building_size'] / (df['land_size'] + 1)).clip(0, 1).astype('float32')
    df['unused_land'] = (df['land_size'] - df['building_size']).clip(0).astype('float32')

In [ ]:
if all(col in df.columns for col in ['rooms_count', 'building_size']):
    df['room_size_avg'] = (df['building_size'] / (df['rooms_count'] + 1)).astype('float32')
    df['room_density'] = (df['rooms_count'] / (df['building_size'] + 1)).astype('float32')

In [ ]:
# Floor and position features
if all(col in df.columns for col in ['floor', 'total_floors_count']):
    df['floor_ratio'] = (df['floor'] / (df['total_floors_count'] + 1)).clip(0, 1).astype('float32')
    df['is_ground_floor'] = (df['floor'] == 0).astype('int8')
    df['is_top_floor'] = (df['floor'] == df['total_floors_count']).astype('int8')
    df['is_middle_floor'] = ((df['floor'] > 0) & (df['floor'] < df['total_floors_count'])).astype('int8')

In [ ]:
# Price efficiency metrics
for price_col in ['transformable_price', 'price_value']:
    if price_col in df.columns and 'building_size' in df.columns:
        df[f'{price_col}_per_sqm'] = (df[price_col] / (df['building_size'] + 1)).astype('float32')

In [ ]:
for rent_col in ['transformable_rent', 'rent_value']:
    if rent_col in df.columns and 'building_size' in df.columns:
        df[f'{rent_col}_per_sqm'] = (df[rent_col] / (df['building_size'] + 1)).astype('float32')

In [ ]:
# Rental yield calculation (if both price and rent available)
if all(col in df.columns for col in ['transformable_price', 'transformable_rent']):
    df['rental_yield'] = (df['transformable_rent'] * 12 / (df['transformable_price'] + 1) * 100).clip(0, 50).astype('float32')


In [ ]:
# Amenities scoring system
luxury_amenities = ['has_pool', 'has_jacuzzi', 'has_sauna', 'has_barbecue']
comfort_amenities = ['has_elevator', 'has_parking', 'has_balcony', 'has_warehouse']
basic_amenities = ['has_water', 'has_gas', 'has_electricity']
security_amenities = ['has_security_guard']

In [ ]:
for amenity_group, name in [(luxury_amenities, 'luxury'),
                           (comfort_amenities, 'comfort'),
                           (basic_amenities, 'basic'),
                           (security_amenities, 'security')]:
    available_amenities = [col for col in amenity_group if col in df.columns]
    if available_amenities:
        df[f'{name}_score'] = df[available_amenities].sum(axis=1).astype('int8')


In [ ]:
# Total amenities score with weights
amenity_weights = {'luxury_score': 3, 'comfort_score': 2, 'basic_score': 1, 'security_score': 2}
df['total_amenity_score'] = 0
for score_col, weight in amenity_weights.items():
    if score_col in df.columns:
        df['total_amenity_score'] += df[score_col] * weight
df['total_amenity_score'] = df['total_amenity_score'].astype('Int8')


In [ ]:
# Location desirability (based on neighborhood price statistics)
if 'neighborhood_slug' in df.columns and 'transformable_price' in df.columns:
    neighborhood_stats = df.groupby('neighborhood_slug')['transformable_price'].agg(['mean', 'median', 'count'])
    neighborhood_stats['price_rank'] = neighborhood_stats['median'].rank(pct=True)
    df = df.merge(neighborhood_stats[['price_rank']].reset_index(), on='neighborhood_slug', how='left')
    df['neighborhood_desirability'] = (df['price_rank'] * 5).round().astype('int8')
    df.drop('price_rank', axis=1, inplace=True)

In [ ]:
# Market segment classification
if 'transformable_price' in df.columns:
    # Ensure numeric
    df['transformable_price'] = pd.to_numeric(df['transformable_price'], errors='coerce')

    # Drop NaNs for quantile calculation
    valid_prices = df['transformable_price'].dropna()
    if not valid_prices.empty:
        price_quartiles = valid_prices.quantile([0.25, 0.5, 0.75])

        # Define bins safely
        bins = [0] + price_quartiles.tolist() + [float('inf')]

        # Ensure bins are strictly increasing
        bins = sorted(set(bins))

        # Create labels dynamically
        labels = ['Budget', 'Mid_Range', 'Premium', 'Luxury'][:len(bins)-1]

        df['market_segment'] = pd.cut(
            df['transformable_price'],
            bins=bins,
            labels=labels,
            duplicates='drop'   # handle duplicate edges safely
        ).astype(str)


# =============================================================================
# STEP 6:  CATEGORICAL ENCODING
# =============================================================================

In [ ]:
# Identify categorical columns
categorical_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()

# Target encoding for high-cardinality location columns (if target available)
high_card_cols = ['city_slug', 'neighborhood_slug']
target_col = 'transformable_price'  # Primary target

In [47]:
if target_col in df.columns:
    for col in high_card_cols:
        if col in df.columns and col in categorical_cols:
            # Calculate mean target value for each category
            target_mean = df.groupby(col)[target_col].mean()
            global_mean = df[target_col].mean()

            # Add smoothing to prevent overfitting
            category_counts = df[col].value_counts()
            smoothing = 100  # Smoothing parameter
            smoothed_means = (target_mean * category_counts + global_mean * smoothing) / (category_counts + smoothing)

            df[f'{col}_target_encoded'] = df[col].map(smoothed_means).fillna(global_mean).astype('float32')
            categorical_cols.remove(col) if col in categorical_cols else None

NameError: name 'target_col' is not defined

In [ ]:
# Label encoding for remaining categorical columns
label_encoders = {}
for col in categorical_cols:
    if col in df.columns:
        le = LabelEncoder()
        # Handle missing values
        df[col] = df[col].astype(str).fillna('Unknown')
        df[col] = le.fit_transform(df[col])
        label_encoders[col] = le

        # Convert to appropriate integer type
        if len(le.classes_) < 128:
            df[col] = df[col].astype('Int8')
        elif len(le.classes_) < 32768:
            df[col] = df[col].astype('Int16')

print(f"Encoded {len(categorical_cols)} categorical columns")

In [ ]:
# =============================================================================
# STEP 7: FEATURE SCALING AND NORMALIZATION
# =============================================================================

In [ ]:
# Identify numerical columns for scaling
numerical_cols = df.select_dtypes(include=[np.number]).columns.tolist()
price_cols_in_df = [col for col in price_columns if col in numerical_cols]
size_cols_in_df = [col for col in size_columns if col in numerical_cols]

In [ ]:
# Different scaling strategies for different types of features
scalers = {}

# Log transformation for highly skewed price data
for col in price_cols_in_df:
    if df[col].min() > 0:  # Ensure all values are positive
        df[f'{col}_log'] = np.log1p(df[col]).astype('float32')

In [ ]:
# Power transformation for moderately skewed data
power_transformer = PowerTransformer(method='yeo-johnson', standardize=True)
skewed_cols = []
for col in numerical_cols:
    if col not in price_cols_in_df and df[col].dtype in ['float32', 'float64']:
        skewness = abs(df[col].skew())
        if skewness > 1:  # Moderately skewed
            skewed_cols.append(col)

if skewed_cols:
    df_skewed_transformed = power_transformer.fit_transform(df[skewed_cols])
    for i, col in enumerate(skewed_cols):
        df[f'{col}_transformed'] = df_skewed_transformed[:, i].astype('float32')
    scalers['power_transformer'] = power_transformer

In [ ]:
# Robust scaling for price and size features (resistant to outliers)
robust_scaler = RobustScaler()
robust_cols = price_cols_in_df + size_cols_in_df + [col for col in numerical_cols if 'per_sqm' in col]
robust_cols = [col for col in robust_cols if col in df.columns]

if robust_cols:
    df_robust_scaled = robust_scaler.fit_transform(df[robust_cols])
    for i, col in enumerate(robust_cols):
        df[f'{col}_robust_scaled'] = df_robust_scaled[:, i].astype('float32')
    scalers['robust_scaler'] = robust_scaler

In [ ]:
# Standard scaling for normally distributed features
normal_cols = [col for col in numerical_cols
               if col not in robust_cols and col not in skewed_cols
               and col not in price_cols_in_df
               and df[col].dtype in ['float32', 'float64']
               and abs(df[col].skew()) <= 1]

if normal_cols:
    standard_scaler = StandardScaler()
    df_standard_scaled = standard_scaler.fit_transform(df[normal_cols])
    for i, col in enumerate(normal_cols):
        df[f'{col}_standard_scaled'] = df_standard_scaled[:, i].astype('float32')
    scalers['standard_scaler'] = standard_scaler

print(f"Applied scaling to {len(robust_cols + normal_cols + skewed_cols)} numerical features")

In [ ]:

# =============================================================================
# STEP 8: FEATURE SELECTION AND DIMENSIONALITY REDUCTION
# =============================================================================

In [ ]:
# Remove highly correlated features to reduce multicollinearity
correlation_threshold = 0.85
numerical_features = df.select_dtypes(include=[np.number]).columns.tolist()

if len(numerical_features) > 1:
    corr_matrix = df[numerical_features].corr().abs()
    upper_triangle = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

    # Find highly correlated feature pairs
    high_corr_pairs = [(corr_matrix.index[i], corr_matrix.columns[j])
                       for i in range(len(corr_matrix.index))
                       for j in range(len(corr_matrix.columns))
                       if upper_triangle.iloc[i, j] > correlation_threshold]

    # Remove features with high correlation (keep the first one in each pair)
    features_to_drop = []
    for feature1, feature2 in high_corr_pairs:
        if feature2 not in features_to_drop:
            features_to_drop.append(feature2)

    if features_to_drop:
        df.drop(columns=features_to_drop, inplace=True)
        print(f"Removed {len(features_to_drop)} highly correlated features")

In [ ]:
# =============================================================================
# STEP 9: MEMORY OPTIMIZATION
# =============================================================================

In [ ]:
# Optimize data types
for col in df.columns:
    if df[col].dtype == 'float64':
        df[col] = pd.to_numeric(df[col], downcast='float')
    elif df[col].dtype == 'int64':
        df[col] = pd.to_numeric(df[col], downcast='integer')

# Convert low-cardinality object columns to category
for col in df.select_dtypes(include=['object']).columns:
    if df[col].nunique() < len(df) * 0.3:  # Less than 30% unique values
        df[col] = df[col].astype('category')


In [ ]:
# =============================================================================
# STEP 10: FINAL VALIDATION AND EXPORT
# =============================================================================

In [ ]:
# Ensure no missing values remain
remaining_missing = df.isnull().sum().sum()
if remaining_missing > 0:
    print(f"WARNING: {remaining_missing} missing values remain!")
    # Fill any remaining missing values
    for col in df.columns:
        if df[col].isnull().sum() > 0:
            if df[col].dtype in ['float16', 'float32', 'float64']:
                df[col] = df[col].fillna(df[col].median())
            else:
                df[col] = df[col].fillna(0)

In [ ]:
# Validate data integrity
assert df.isnull().sum().sum() == 0, "Missing values still exist!"
assert df.shape[0] > 0, "No data remaining!"
assert not df.duplicated().any(), "Duplicate rows exist!"

In [ ]:
# Export processed data
df.to_csv("real_estate_ml_ready.csv", index=False)
df.to_parquet("real_estate_ml_ready.parquet", index=False)
df.to_pickle("real_estate_ml_ready.pkl")

In [ ]:
# Export preprocessing artifacts
import pickle
with open('preprocessing_artifacts.pkl', 'wb') as f:
    pickle.dump({
        'label_encoders': label_encoders,
        'scalers': scalers,
        'feature_columns': df.columns.tolist(),
        'outlier_summary': outlier_summary
    }, f)

In [ ]:
# =============================================================================
# COMPREHENSIVE SUMMARY
# =============================================================================

In [ ]:
end_time = datetime.now()
processing_time = (end_time - start_time).total_seconds()
final_memory = df.memory_usage(deep=True).sum() / 1024**2
memory_reduction = ((original_memory - final_memory) / original_memory) * 100


In [ ]:
print("\n" + "=" * 80)
print("ADVANCED PREPROCESSING COMPLETED SUCCESSFULLY!")
print("=" * 80)
print(f"Original shape: {original_shape[0]:,} × {original_shape[1]}")
print(f"Final shape: {df.shape[0]:,} × {df.shape[1]}")
print(f"Features created: {df.shape[1] - original_shape[1]}")
print(f"Memory usage: {original_memory:.1f} MB → {final_memory:.1f} MB ({memory_reduction:.1f}% reduction)")
print(f"Processing time: {processing_time:.1f} seconds")
print(f"Missing values: {remaining_missing}")

In [ ]:

print(f"\nData Quality Improvements:")
print(f"  ✓ Advanced outlier detection and capping")
print(f"  ✓ Multi-strategy missing value imputation")
print(f"  ✓ Sophisticated feature engineering ({df.shape[1] - original_shape[1]} new features)")
print(f"  ✓ Domain-specific real estate transformations")
print(f"  ✓ Multi-method outlier handling")
print(f"  ✓ Advanced categorical encoding (target + label)")
print(f"  ✓ Intelligent feature scaling (robust + standard + power)")
print(f"  ✓ Multicollinearity reduction")
print(f"  ✓ Memory optimization")

In [ ]:
print(f"\nKey Real Estate Features Created:")
new_features = [
    'building_age', 'age_category', 'condition_score',
    'building_to_land_ratio', 'room_size_avg', 'room_density',
    'floor_ratio', 'is_ground_floor', 'is_top_floor',
    'luxury_score', 'comfort_score', 'total_amenity_score',
    'neighborhood_desirability', 'market_segment', 'rental_yield'
]

for feature in new_features:
    if feature in df.columns:
        print(f"  ✓ {feature}")

print(f"\nData Types Summary:")
dtype_counts = df.dtypes.value_counts()
for dtype, count in dtype_counts.items():
    print(f"  {dtype}: {count} columns")

In [ ]:
print(f"\nTop Features by Type:")
price_features = [col for col in df.columns if 'price' in col.lower() or 'rent' in col.lower()]
size_features = [col for col in df.columns if 'size' in col.lower() or 'area' in col.lower()]
location_features = [col for col in df.columns if any(x in col.lower() for x in ['city', 'neighborhood', 'location'])]
amenity_features = [col for col in df.columns if 'has_' in col.lower() or 'score' in col.lower()]

print(f"  Price-related: {len(price_features)} features")
print(f"  Size-related: {len(size_features)} features")
print(f"  Location-related: {len(location_features)} features")
print(f"  Amenity-related: {len(amenity_features)} features")

In [ ]:
print(f"\nReady for Machine Learning:")
print(f"  ✓ No missing values")
print(f"  ✓ Optimized data types")
print(f"  ✓ Scaled numerical features")
print(f"  ✓ Encoded categorical features")
print(f"  ✓ Rich feature set for real estate prediction")
print(f"  ✓ Preprocessing artifacts saved for production use")